# MSCA 31008 - Data Mining Assignment 4 Part 1 (Group 4)
<b>Qingwei Zhang, Jake Brewer, Prinu Mathew</b><br>
<b>Winter 2023</b>

### Import Libraries  

In [ ]:
import sys, os, json, subprocess

## for data
import pandas as pd
import numpy as np
import datetime
import random

## for machine learning
from scipy import stats

import warnings

warnings.filterwarnings("ignore")


In [ ]:
## for machine learning
try:
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import (
        confusion_matrix,
        classification_report,
        f1_score,
        roc_auc_score,
        roc_curve,
        accuracy_score,
    )
    import sklearn.datasets

    print("~~~ Already installed required packages for machine learning ~~~~")
except Exception as e:
    print(e)

    print("~~~ Installing required packages for machine learning ~~~~")
    subprocess.check_call(
        [sys.executable, "-m", "pip", "install", "--upgrade", "kneed"]
    )
    subprocess.check_call(
        [sys.executable, "-m", "pip", "install", "--upgrade", "scikit-learn"]
    )
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import (
        confusion_matrix,
        classification_report,
        f1_score,
        roc_auc_score,
        roc_curve,
        accuracy_score,
    )
    import sklearn.datasets


In [ ]:
## for interactive visualization
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    print("~~~ Already installed required packages for interactive visualizations ~~~~")
except Exception as e:
    print(e)
    
    print("~~~ Installing required packages for interactive visualizations ~~~~")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "matplotlib"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "seaborn"])
    import matplotlib.pyplot as plt
    import seaborn as sns
    
%matplotlib inline

## 1. Read dataset

In [ ]:
# import data and read csv

input_df = pd.read_csv("diabetes_data.csv")
input_df.head()


In [ ]:
# view data types and number of non-null values in each column
input_df.info()


## 2. Prepare Data for Input into Logistic Regression Model

In [ ]:
# convert 2 value categories into binary variables
input_df["readmitted"] = input_df["readmitted"].replace(">30", "YES")
input_df["readmitted"] = input_df["readmitted"].replace("<30", "YES")
input_df["readmitted"] = input_df["readmitted"].replace("YES", 1)
input_df["readmitted"] = input_df["readmitted"].replace("NO", 0)
input_df["readmitted"] = input_df["readmitted"].astype(int)
input_df["readmitted"].value_counts()


#### diag_1, diag_2, diag_3 variables : 

The dataset contained upto three diagnoses for a given patient (primary (diag_1), secondary(diag_2) and additional(diag_3)). However, each of these had 700–900 unique ICD codes  
We can collapsed these diagnosis codes into 9 disease categories . These 9 categories include Circulatory, Respiratory, Digestive, Diabetes, Injury, Musculoskeletal, Genitourinary, Neoplasms, and Others. 

* You can convert all 3 into 9 categories each. however primary diagnosis is enough for this study.

ICD9 code conversion online reference (https://en.wikipedia.org/wiki/List_of_ICD-9_codes)

In [ ]:
# recategorize diagnoses into 9 categories - only consider primary diagnosis
def convert_diag_codes(code):
    if pd.isnull(code):
        return "Other"
    elif ("V" in code) or ("E" in code):
        return "Other"
    else:
        code = float(code)
        if (code >= 390) and (code < 460) or (np.floor(code) == 785):
            return "Circulatory"
        elif (code >= 460) and (code < 520) or (np.floor(code) == 786):
            return "Respiratory"
        elif (code >= 520) and (code < 580) or (np.floor(code) == 787):
            return "Digestive"
        elif code == 250:
            return "Diabetes"
        elif (code >= 800) and (code < 1000):
            return "Injury"
        elif (code >= 710) and (code < 740):
            return "Musculoskeletal"
        elif (code >= 580) and (code < 630) or (np.floor(code) == 788):
            return "Genitourinary"
        elif (code >= 140) and (code < 240):
            return "Neoplasms"
        else:
            return "Other"


input_df["diag_1"] = input_df["diag_1"].apply(lambda x: convert_diag_codes(x))
input_df["diag_1"].value_counts()


In [ ]:
# convert age to numeric
age_dict = {
    "[0-10)": 5,
    "[10-20)": 15,
    "[20-30)": 25,
    "[30-40)": 35,
    "[40-50)": 45,
    "[50-60)": 55,
    "[60-70)": 65,
    "[70-80)": 75,
    "[80-90)": 85,
    "[90-100)": 95,
}

input_df["age"] = input_df["age"].map(age_dict)
input_df["age"] = input_df["age"].astype("int64")


In [ ]:
# convert 2 value categories into binary variables
input_df["change"] = input_df["change"].replace("Ch", 1)
input_df["change"] = input_df["change"].replace("No", 0)
input_df["gender"] = input_df["gender"].replace("Male", 1)
input_df["gender"] = input_df["gender"].replace("Female", 0)
input_df["diabetesMed"] = input_df["diabetesMed"].replace("Yes", 1)
input_df["diabetesMed"] = input_df["diabetesMed"].replace("No", 0)


In [ ]:
# merge categories to 1 - 2 levels using domain knowledge
input_df["A1Cresult"] = input_df["A1Cresult"].replace(">7", "Abnormal")
input_df["A1Cresult"] = input_df["A1Cresult"].replace(">8", "Abnormal")
input_df["A1Cresult"] = input_df["A1Cresult"].replace("Norm", "Normal")
input_df["A1Cresult"] = input_df["A1Cresult"].replace("None", "Not tested")
input_df["max_glu_serum"] = input_df["max_glu_serum"].replace(">200", "Abnormal")
input_df["max_glu_serum"] = input_df["max_glu_serum"].replace(">300", "Abnormal")
input_df["max_glu_serum"] = input_df["max_glu_serum"].replace("Norm", "Normal")
input_df["max_glu_serum"] = input_df["max_glu_serum"].replace("None", "Not tested")


In [ ]:
# merge some categories together
input_df["admission_type_id"] = input_df["admission_type_id"].replace(2, 1)
input_df["admission_type_id"] = input_df["admission_type_id"].replace(7, 1)
input_df["admission_type_id"] = input_df["admission_type_id"].replace(6, 5)
input_df["admission_type_id"] = input_df["admission_type_id"].replace(8, 5)


In [ ]:
# convert numerical look-a-likes to string
input_df["admission_type_id"] = input_df["admission_type_id"].map(str)
input_df["discharge_disposition_id"] = input_df["discharge_disposition_id"].map(str)


In [ ]:
# remove duplicate patients
input_df = input_df.drop_duplicates(subset=["patient_nbr"], keep="first")


In [ ]:
# drop/remove unnecessary variables
input_df.drop(
    ["patient_nbr", "diag_2", "diag_3", "encounter_id", "admission_source_id"],
    axis=1,
    inplace=True,
)


In [ ]:
# fill missing value for race
input_df["race"].replace(np.nan, "Missing", inplace=True)


## 3. Apply one-hot encoding

In [ ]:
# apply one-hot encoding
one_hot_list = [col for col in input_df.columns if input_df[col].dtype == "object"]
data_preprocess = pd.get_dummies(input_df, columns=one_hot_list, drop_first=True)
data_preprocess.info()


In [ ]:
# save dataframe to csv for future use in part 2
data_preprocess.to_csv("diabetes_data_preprocess.csv", index=False)


After pre-processing the data we are left with 102 columns (original dataset had 44) and data on 68,630 unique patients (removed duplicate patient data, keeping first entry).

## 4. Split Data into Training (70%) and Testing (30%)

In [ ]:
# split the data into features and target and using random state so that results are reproducible in part 2

X = data_preprocess.drop(columns=["readmitted"])
y = data_preprocess["readmitted"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)
y_train.value_counts()


## 5. Train Model on All Variables to Find Most Important

-   Use sklearn.linear_model.LogisticRegression OR discrete.discrete_model
-   Use cross validation, parameter tuning (penalty, c) using GridSearchCV
-   Select best variables by looking at coefficients of variables and fit model with best variables and best parameters

In [ ]:
# specify class_weight='balanced' to account for imbalance in re-admitted and non re-admitted patients
clf = LogisticRegression(class_weight="balanced", random_state=42, max_iter=100)
clf.fit(X_train, y_train)

# predict on the test set
y_pred = clf.predict(X_test)

# evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Actual Accuracy:", accuracy)


In [ ]:
# run grid search cross validation to find best regularization hyper-parameters
param_map = {"penalty": ["l1", "l2"], "C": [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

clf_gs = GridSearchCV(clf, param_grid=param_map, cv=5, verbose=1)
clf_gs.fit(X_train, y_train)


In [ ]:
pd.DataFrame(
    clf_gs.cv_results_,
    columns=[
        "rank_test_score",
        "param_C",
        "param_penalty",
        "mean_test_score",
        "std_test_score",
    ],
).sort_values(by=["rank_test_score"])


In [ ]:
# identify best hyper-parameters
best_penalty = clf_gs.best_params_["penalty"]
best_c = clf_gs.best_params_["C"]
clf_gs.best_params_


In [ ]:
# use the best estimator from the cross validation to identify the most important features based on their coefficients
best_model = clf_gs.best_estimator_
coef_df = pd.DataFrame(
    {"Variable": X_train.columns, "Coefficient": best_model.coef_.flatten()}
)
coef_df["Abs Coefficient"] = abs(coef_df["Coefficient"])
sorted_coef_df = coef_df.sort_values(by="Abs Coefficient", ascending=False)
sorted_coef_df.head(50)


In [ ]:
# choose the top 40 features with highest coefficient
best_coefs = sorted_coef_df["Variable"].iloc[:40]
X_train_best_features = X_train[best_coefs]
X_test_best_features = X_test[best_coefs]

X_train_best_features


In [ ]:
# train new model with best features
clf_trim = LogisticRegression(
    class_weight="balanced",
    C=best_c,
    penalty=best_penalty,
    random_state=42,
    max_iter=100,
)
clf_trim.fit(X_train_best_features, y_train)


In [ ]:
# predict on the test set
y_pred = clf_trim.predict(X_test_best_features)

# evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy after cross validation:", accuracy)


In [ ]:
f1_score(y_train, clf.predict(X_train))


In [ ]:
f1_score(y_train, best_model.predict(X_train))


In [ ]:
f1_score(y_train, clf_trim.predict(X_train_best_features))


Note: A good F1 score (the harmonic mean of precision and recall) means that you have low false positives and low false negatives, so you're correctly identifying real threats and you are not disturbed by false alarms. An F1 score is considered perfect when it's 1 , while the model is a total failure when it's 0

In [ ]:
# compare the results of model with all features to model with fewer features
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].bar(
    ["All Predictors", "Best Predictors"],
    [
        best_model.score(X_train, y_train),
        clf_trim.score(X_train_best_features, y_train),
    ],
    color=["b", "r"],
)

ax[1].bar(
    ["All Predictors", "Best Predictors"],
    [
        f1_score(y_train, best_model.predict(X_train)),
        f1_score(y_train, clf_trim.predict(X_train_best_features)),
    ],
    color=["b", "r"],
)

ax[0].set(title="Accuracy", ylabel="Score")
ax[1].set(title="F1-Score", ylabel="Score")

print(f"All Predictor Variables:  {len(best_model.coef_[0, :])}")
print(f"Best Predictor Variables: {len(clf_trim.coef_[0, :])}")

plt.show()


After running grid search cross validation we identified the best C hyper-tuning parameter to be 10 with l2 regularization. Using these hyper-tuning parameters we identified the top 40 features that had the most significant impact on the model based on their coefficients. We re-trained the logistic regression model using only these 40 features and compared the accuracy and F1-score to that of the original model with 101 variables. The accuracy of the second model on the trained data is about the same as the accuracy of the original model, while the F1-score of the second model is about .02 higher than the F1-score of the original model. Due to this slight increase in performance we will stick with the second model (40 features) and moving forward.

## 6. Analyze Model Performance on Training Data

In [ ]:
# make predictions for training data

y_pred = clf_trim.predict(X_train_best_features)
# y_pred = best_model.predict(X_train)

# create confusion matrix and classification report
fig, ax = plt.subplots(figsize=(10, 6))
mat = confusion_matrix(y_train, y_pred)
sns.heatmap(
    mat,
    square=True,
    annot=True,
    fmt="d",
    cmap="coolwarm",
    xticklabels=[0, 1],
    yticklabels=[0, 1],
)
ax.set(xlabel="Predicted Label")
ax.set(ylabel="True Label")
print("Threshold = 0.5")
plt.show()
print(classification_report(y_train, y_pred))


In [ ]:
y_pred = np.where(clf_trim.predict_proba(X_train_best_features)[:, 1] > 0.4, 1, 0)
# y_pred = np.where(best_model.predict_proba(X_train)[:,1] > .4, 1, 0)

# create confusion matrix

fig, ax = plt.subplots(figsize=(10, 6))
mat = confusion_matrix(y_train, y_pred)
sns.heatmap(
    mat,
    square=True,
    annot=True,
    fmt="d",
    cmap="coolwarm",
    xticklabels=[0, 1],
    yticklabels=[0, 1],
)
ax.set(xlabel="Predicted Label")
ax.set(ylabel="True Label")
print("Threshold = 0.4")
plt.show()
print(classification_report(y_train, y_pred))


In [ ]:
y_pred = np.where(clf_trim.predict_proba(X_train_best_features)[:, 1] > 0.6, 1, 0)
# y_pred = np.where(best_model.predict_proba(X_train)[:,1] > .6, 1, 0)

# create confusion matrix

fig, ax = plt.subplots(figsize=(10, 6))
mat = confusion_matrix(y_train, y_pred)
sns.heatmap(
    mat,
    square=True,
    annot=True,
    fmt="d",
    cmap="coolwarm",
    xticklabels=[0, 1],
    yticklabels=[0, 1],
)
ax.set(xlabel="Predicted Label")
ax.set(ylabel="True Label")
print("Threshold = 0.6")
plt.show()
print(classification_report(y_train, y_pred))


After analyzing model performance on the training data at various threshold, it appears that a 0.6 threshold yields a slightly higher accuracy than the default 0.5 threshold:
- Threshold: 0.5 Accuracy: .60
- Threshold: 0.4 Accuracy: .50
- Threshold: 0.6 Accuracy: .63

But at the cost of a much lower F1-score:
- Threshold: 0.5 F1-Score: .54
- Threshold: 0.4 F1-Score: .59
- Threshold: 0.6 F1-Score: .33

May be business analyst who has domain knowledge could determine which is more important here - catching more at-risk patients for re-admission at the cost of a larger false positive rate, or maximizing overall accuracy. Given the slight decline in accuracy for a much higher F1-score though, we will consider the 0.5 threshold to be the best performer.

## 7. Analyze Model Performance on Testing Data

In [ ]:
# make predictions for testing data

y_pred = clf_trim.predict(X_test_best_features)

# create confusion matrix and classification report
fig, ax = plt.subplots(figsize=(6, 5))
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(
    mat,
    square=True,
    annot=True,
    fmt="d",
    cmap="coolwarm",
    xticklabels=[0, 1],
    yticklabels=[0, 1],
)
ax.set(xlabel="Predicted Label")
ax.set(ylabel="True Label")
plt.show()
print(classification_report(y_test, y_pred))


The trimmed model is more balanced between precision and recall. However the positive class (readmitted=1) still suffers from poor performance. Namely, there is only a 50% (precision=0.50) chance that the patient actually needs to be readmitted when the model says the patient should be admitted. Perhaps it's better that recall is higher than precision because it's better to be safe than sorry and be better at identifying positive cases

## 8. Plot ROC Curve

In [ ]:
# create ROC-AUC curve

roc_auc = roc_auc_score(y_test, y_pred)
fpr, tpr, thresholds = roc_curve(
    y_test, clf_trim.predict_proba(X_test_best_features)[:, 1]
)

fig, ax = plt.subplots(figsize=(6, 5))
ax.plot(fpr, tpr, label=f"Logistic Regression Area: {roc_auc.round(4)}")
ax.plot([0, 1], [0, 1], "r--")
ax.set(
    xlim=[0.0, 1.0],
    ylim=[0.0, 1.0],
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title="Receiver Operating Characteristic",
)
ax.legend(loc="lower right")
plt.show()


According to the ROC curve, our model is slightly better than random chance

## 9. Summarize Results

### Performance on Test Dataset
The accuracy of the logistic regression model on the test data is 60% with a .53 F1-score. The breakdown of readmitted patients on the total dataset is 40.7% readmitted and 59.3% not readmitted. Therefore the results of our logistic regression model are barely better than simply guessing that all patients will not have to be readmitted (59.3% accuracy) and it is a better than randomly guessing (50% accuracy) as seen in the ROC-AUC graph.

By specifying weight class imbalance when training our model we were greatly able to even out the errors between false positives and false negatives as opposed to overwehlmingly guessing the majority class and having a higher false negative rate. This led to a higher F1-score, but did not have much of an impact on accuracy.

### Model Performance on Train vs Test Datasets
A positive for the model is that in all scenarios the training results and testing results were very close together, implying that the model is likely not overfitting, but might be underfitting. We ran grid search on 2 regularization hyper-parameters (C and penalty) which both decrease the chance that the model will overfit the training data. The confusion matrices and classification reports for the training data and testing data was very similar implying stability in the model.

### Final Thoughts
Despite a decent amount of pre-processing, the logistic regression model performed quite poorly on predicting which patients are at risk of re-admission. Additional feature engineering with the help of domain expertise could possibly improve accuracy, but most likely logistic regression is not the right model for this problem.

In [ ]:
import datetime
import pytz

datetime.datetime.now(pytz.timezone("US/Central")).strftime("%a, %d %B %Y %H:%M:%S")
